In [2]:
import os 

In [3]:
%pwd

'c:\\code\\ML\\time_series_forecast\\time-series-forecast\\research'

In [4]:
os.chdir('../')

In [5]:
%pwd

'c:\\code\\ML\\time_series_forecast\\time-series-forecast'

In [7]:
from dataclasses import dataclass
from pathlib import Path
@dataclass 
class Prediction:
   auto_arima : Path
   prophet : Path
   

In [8]:
#configuration

In [9]:
from src.time_series.constants import *
from src.time_series.utils.common import create_directories , read_yaml
from src.time_series.entity.config_entity import DataIngestionconfig,EDA,Model,Prediction



class ConfigurationManager:
    def __init__(self,config_file_path=CONFIG_FILE_PATH,params_file_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)->DataIngestionconfig:
        config = self.config.data_ingestion
        data_ingestion_config = DataIngestionconfig(root_dir= config.root_dir,source_url=config.source_url,local_data_file=config.local_data_file)
        return data_ingestion_config
    def get_eda_config(self)->EDA:
        config = self.config.EDA
        create_directories([config.report_path])
        create_directories([config.data_output])
        eda_config = EDA(data_path= config.data_path,report_path=config.report_path,data_output=config.data_output)
        
        return eda_config
    def get_model_config(self)->Model:
        config = self.config.model
        create_directories([config.report])
        model_config = Model(data_path=config.data_path,report=config.report,model_save_path= config.model_save_path)
        return model_config
    def get_prediction_config(self)-> Prediction:
        config = self.config.prediction
        prediction_config = Prediction(auto_arima= config.auto_arima , prophet= config.prophet)
        return prediction_config 

In [10]:
#component

In [13]:
import pandas as pd
import joblib
import os
import matplotlib.pyplot as plt
import io
from prophet import Prophet
from flask import send_file

class PredictionComponent:
    def __init__(self,config = Prediction):
        self.config = config

    def prediction_with_auto_arima(self,data:pd.DataFrame , model_path : Path, periods : int):
        model = joblib.load(model_path)
        forecast = model.predict(n_periods = periods)

        # Generate future index
        last_date = data.index[-1]
        freq = pd.infer_freq(data.index) or 'MS'
        forecast_index = pd.date_range(start=last_date + pd.tseries.frequencies.to_offset(freq),
                                    periods=periods, freq=freq)
        forecast_series = pd.Series(forecast, index=forecast_index)

        # Plot
        plt.figure(figsize=(10, 5))
        plt.plot(data, label="Historical")
        plt.plot(forecast_series, label="Forecast", color='orange')
        plt.xlabel("Date")
        plt.ylabel("Value")
        plt.title("Auto ARIMA Forecast")
        plt.legend()
        plt.grid(True)

        # Save to in-memory buffer
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        plt.clf()

        # Return image as response
        return send_file(buf, mimetype='image/png')
    


    def prediction_with_prophet(self,data: pd.DataFrame, periods: int):
        # Prepare the data
        df = data.reset_index().rename(columns={'Month': 'ds', 'passengers': 'y'})

        # Train the model
        model = Prophet(weekly_seasonality=False, yearly_seasonality=True, changepoint_prior_scale=0.01)
        model.fit(df)

        # Make future dataframe and forecast
        future = model.make_future_dataframe(periods=periods, freq='MS')
        forecast = model.predict(future)

        # Plot forecast
        fig = model.plot(forecast)
        fig.suptitle("Prophet Forecast", fontsize=16)
        plt.xlabel("Date")
        plt.ylabel("Passengers")

        # Save plot to in-memory buffer
        buf = io.BytesIO()
        fig.savefig(buf, format='png')
        buf.seek(0)
        plt.close(fig)

        return buf
            